
<div align="center">
  <img src="https://www.dropbox.com/s/vold2f3fm57qp7g/ECE4179_5179_6179_banner.png?dl=1" alt="ECE4179/5179/6179 Banner" style="max-width: 60%;"/>
</div>

<div align="center">

# Recurrent Neural Networks (RNNs)

</div>

Welcome to week 10 of ECE4179/5179/6179! 

*Dun dun dun dun-da-da dun-da-da, dun-da-da!* 🎶

In an island far, far away you find yourself on a distant jungle, with nothing but your trusty laptop, equipped with the latest release of PyTorch and other beloved Python packages. 

What would you do? Of course you start coding and learning about **Recurrent Neural Networks (RNNs)**!
Let’s get started, May the gradients be with you! ✨


<div align="center">
  <img src="data/Mysterious-Island.jpg" alt="mysterious_island" style="max-width: 60%;"/>
</div>



In [12]:
import os
import random


import csv
import numpy as np
import pandas as pd
from tqdm import tqdm


import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.metrics import mean_squared_error


import torch
from torch import nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader




from PIL import Image

RND_SEED = 42
np.random.seed(RND_SEED)
random.seed(RND_SEED)
torch.manual_seed(RND_SEED)


# Check if CUDA is available (for NVIDIA GPUs)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA GPU detected. Using CUDA.")

# Check if MPS (Metal Performance Shaders) is available (for Apple Silicon M1/M2)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Apple Silicon GPU detected. Using MPS.")

# Otherwise, default to CPU
else:
    device = torch.device("cpu")
    print("No GPU available. Using CPU.")

No GPU available. Using CPU.


<div style="background-color: #2b0080; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #1. So you claim to be a Jedi in neural networks?</span>

Over the past 10 weeks, you've achieved amazing things with neural networks. Now, I challenge you to design an MLP for a simple task: adding two 3-digit numbers. For example, given 4179 and 5179, your MLP should return their sum, 9358. To simplify things further, if the sum exceeds 9999, just return the last 4 digits (e.g., 899 + 101 = 000).

Consider the input to your model as 2x3 symbols (digits), where each symbol can take 10 possible values (0 to 9), and the output will be 3 symbols. Discuss how you would approach this as a supervised learning problem with an MLP. Think about how to design the output, handle classification (if needed), process the input, and other aspects of your solution.

</div>


<div style="background-color: #2b0080; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #2. Have a go at your MLP!</span>

Great! Hopefully, you have a solid plan for Task #1. Now, let's implement your MLP in PyTorch. Below, I’ve provided a couple of functions to help prepare the data and evaluate the performance of your model. Your task is to design the model and the training loop. 

If you're unsure how to approach this problem, here's a suggestion to get started:

The input to your MLP should be an 8D vector—where the first 4 elements represent the digits of the first number, and the last 4 represent the second number. Instead of formulating this as a regression problem (which would be inefficient since the output space isn't continuous), treat it as a classification problem. Each digit is a class, meaning you'll need 4x10 output units. Use the cross-entropy loss function, applying it separately to each 10D output vector to predict each digit. PyTorch’s `CrossEntropyLoss` handles this directly.

</div>


In [2]:
class DigitAdditionDataset(Dataset):
    """
    A PyTorch Dataset class to generate pairs of random n-digit numbers and their sums, 
    where the sum is truncated or padded to match the length of the input sequences.

    Args:
        size (int): The total number of samples in the dataset. Default is 1000.
        seq_len (int): Length of the input sequences (i.e., number of digits in each number). Default is 4.
    
    Returns:
        A tuple of three elements:
            - seq1 (torch.Tensor): The first number represented as a sequence of digits.
            - seq2 (torch.Tensor): The second number represented as a sequence of digits.
            - sum_seq (torch.Tensor): The sum of the two numbers, truncated/padded to seq_len.
    """
    def __init__(self, size=1000, seq_len=3):
        self.seq_len = seq_len  # Length of the sequence (number of digits in the numbers)
        self.size = size  # Total number of data samples to generate
        self.data = []  # Initialize an empty list to store the generated data
        
        # Define the range for random numbers based on the sequence length
        min_val = 10 ** (seq_len - 1)  # Minimum value for a number with seq_len digits (e.g., 100 for 3 digits)
        max_val = (10 ** seq_len) - 1  # Maximum value for a number with seq_len digits (e.g., 999 for 3 digits)
        
        # Generate 'size' number of data samples
        for _ in range(size):
            num1 = random.randint(min_val, max_val)  # Randomly generate the first number
            num2 = random.randint(min_val, max_val)  # Randomly generate the second number
            sum_result = num1 + num2  # Calculate the sum of the two numbers
            
            # Convert the numbers into lists of digits
            seq1 = [int(digit) for digit in str(num1)]  # Convert num1 to a list of digits
            seq2 = [int(digit) for digit in str(num2)]  # Convert num2 to a list of digits
            sum_seq = [int(digit) for digit in str(sum_result)]  # Convert sum_result to a list of digits
            
            # Ensure the sequences are all the same length (seq_len) by truncating or padding the sum
            if len(sum_seq) > seq_len:
                sum_seq = sum_seq[-seq_len:]  # Truncate the sum if it has more digits than seq_len
            else:
                sum_seq = [0] * (seq_len - len(sum_seq)) + sum_seq  # Pad the sum with leading zeros if it has fewer digits
            
            # Append the generated sequence (input1, input2, and the sum) to the data list
            self.data.append((seq1, seq2, sum_seq))
    
    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        return self.size
    
    def __getitem__(self, idx):
        """
        Returns the input-output pair (two input sequences and their sum) for a given index.
        
        Args:
            idx (int): Index of the sample to retrieve.
        
        Returns:
            A tuple of torch.Tensors:
                - seq1: The first number represented as a sequence of digits.
                - seq2: The second number represented as a sequence of digits.
                - sum_seq: The sum of the two numbers, represented as a sequence of digits.
        """
        seq1, seq2, sum_seq = self.data[idx]
        return torch.tensor(seq1), torch.tensor(seq2), torch.tensor(sum_seq)


In [3]:
# Training dataset
train_size = 500  # Number of training examples
train_dataset = DigitAdditionDataset(size=train_size)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Test dataset
test_size = 1000  # Number of test examples
test_dataset = DigitAdditionDataset(size=test_size)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [5]:
# Get the first batch from train_loader
seq1, seq2, sum_seq = next(iter(train_loader))

# Print the first 5 data samples from the batch
for i in range(5):
    print(f"Sequence 1: {seq1[i].tolist()}")
    print(f"Sequence 2: {seq2[i].tolist()}")
    print(f"Sum       : {sum_seq[i].tolist()}")
    print("-" * 30)  # Separator


Sequence 1: [9, 8, 2]
Sequence 2: [5, 8, 6]
Sum       : [5, 6, 8]
------------------------------
Sequence 1: [3, 8, 4]
Sequence 2: [2, 5, 9]
Sum       : [6, 4, 3]
------------------------------
Sequence 1: [1, 2, 5]
Sequence 2: [2, 8, 3]
Sum       : [4, 0, 8]
------------------------------
Sequence 1: [4, 8, 8]
Sequence 2: [7, 9, 3]
Sum       : [2, 8, 1]
------------------------------
Sequence 1: [1, 0, 0]
Sequence 2: [7, 1, 3]
Sum       : [8, 1, 3]
------------------------------


In [6]:
def train_addition(model, train_loader, num_epochs=10, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()  # Cross-entropy for classification

    # Initialize progress bar across epochs
    progress_bar = tqdm(total=num_epochs, desc="Training Progress", position=0, leave=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for seq1, seq2, sum_seq in train_loader:
            seq1, seq2, sum_seq = seq1.float(), seq2.float(), sum_seq.long()  # Input in float, target in long (integer)

            optimizer.zero_grad()
            outputs = model(seq1, seq2)  # Shape: seq1:[batch_size, seq_len] seq2:[batch_size, seq_len]
            
            loss = criterion(outputs, sum_seq)  # Compare predicted digit classes with actual sum digit classes
            
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Update the tqdm progress bar
        progress_bar.set_postfix(loss=total_loss / len(train_loader))  # Update the loss on the progress bar
        progress_bar.update(1)  # Update the bar by 1 epoch

    # Close the progress bar when training is done
    progress_bar.close()





In [7]:
# Function to evaluate the MLP classification model on the test dataset
def evaluate_addition(model, test_loader):
    """
    Evaluates the performance of the MLP addition model on the test dataset by calculating
    the number of correctly summed sequences. The function returns the accuracy as a percentage.

    Args:
        model (nn.Module): The trained MLP addition model.
        test_loader (torch.utils.data.DataLoader): DataLoader providing the test dataset.

    Returns:
        float: The accuracy as a percentage of correctly summed sequences.
    
    This function:
    - Sets the model to evaluation mode.
    - Iterates over the test dataset.
    - For each batch, it predicts the digit sequences using the model.
    - Compares the predicted sequences to the target sequences to determine how many sums are correct.
    - Prints the number of correctly summed sequences and the accuracy.
    """
    model.eval()  # Set the model to evaluation mode
    correct_sum = 0  # Counter for correct sum predictions
    total_samples = 0
    
    with torch.no_grad():  # Disable gradient computation for evaluation
        for seq1, seq2, sum_seq in test_loader:
            seq1, seq2, sum_seq = seq1.float(), seq2.float(), sum_seq.long()  # Inputs as float, targets as long

            outputs = model(seq1, seq2)  # Get model predictions, shape: [batch_size, 10, seq_len]
            
            # Get the predicted classes (digit with highest probability)
            predicted_seq = outputs.argmax(dim=1)  # Shape: [batch_size, seq_len]
            
            # Count how many times the entire predicted sequence matches the target sequence
            correct_sum += (predicted_seq == sum_seq).all(dim=1).sum().item()  # Check if all digits match
            total_samples += seq1.size(0)  # Total number of sequences in the batch
    
    # Calculate accuracy as the percentage of correctly predicted sums
    accuracy = correct_sum / total_samples * 100
    print(f"Number of Correctly Summed Sequences: {correct_sum}/{total_samples}, Accuracy: {accuracy:.2f}%")
    
    return accuracy


In [8]:
class MLPAddition(nn.Module):
    """
    A simple Multilayer Perceptron (MLP) model for adding two sequences of digits.
    The model treats the problem as a classification task, where each digit position
    corresponds to one of 10 classes (0-9).

    Args:
        seq_len (int): The length of the input sequences (i.e., number of digits in each sequence).
        hidden_dim (int): The number of hidden units in the fully connected layers.

    Model Structure:
        - Input: Two sequences, each of length `seq_len`.
        - Concatenate the sequences to form an input vector of size `2 * seq_len`.
        - Two hidden layers with ReLU activation.
        - Output layer: `seq_len * 10` units, reshaped to predict a 10-class digit for each position.

    Input:
        - seq1 (torch.Tensor): The first input sequence (batch_size, seq_len).
        - seq2 (torch.Tensor): The second input sequence (batch_size, seq_len).

    Output:
        - Output (torch.Tensor): A tensor of shape (batch_size, seq_len, 10), where the last dimension
                                 represents the 10-class prediction (0-9) for each digit position.
    """
    def __init__(self, seq_len=3, hidden_dim=128):
        super().__init__()
        self.seq_len = seq_len  # Store seq_len for use in the forward method
        self.fc1 = nn.Linear(seq_len * 2, hidden_dim)  # Concatenate seq1 and seq2
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, seq_len * 10)  # Output for each digit position (10 classes per position)
        
    def forward(self, seq1, seq2):
        """
        Forward pass of the MLPAddition model.

        Args:
            seq1 (torch.Tensor): The first input sequence (batch_size, seq_len).
            seq2 (torch.Tensor): The second input sequence (batch_size, seq_len).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, seq_len, 10), where each 10-class vector 
                          represents the predicted digit for each position.
        """
        # Concatenate the sequences along the feature dimension
        x = torch.cat((seq1, seq2), dim=1)  # Shape: [batch_size, seq_len * 2]
        x = torch.relu(self.fc1(x))         # First hidden layer with ReLU activation
        x = torch.relu(self.fc2(x))         # Second hidden layer with ReLU activation
        x = self.fc3(x)                     # Output layer, raw logits for all positions
        x = x.view(-1, 10, self.seq_len)    # Reshape to [batch_size, 10, seq_len] for CrossEntropyLoss
        return x


In [9]:
mlp_addition_model = MLPAddition(seq_len=3, hidden_dim=128)

# Assuming train_loader is already created with the DigitAdditionDataset
train_addition(mlp_addition_model, train_loader, num_epochs=2000, lr=0.01)

# Evaluate the MLP model on the test dataset
acc_trn_mlp_addition = evaluate_addition(mlp_addition_model, train_loader)
acc_tst_mlp_addition = evaluate_addition(mlp_addition_model, test_loader)



Training Progress:   0%|          | 0/2000 [00:00<?, ?it/s]

Training Progress: 100%|██████████| 2000/2000 [03:43<00:00,  8.95it/s, loss=2.03]   


Number of Correctly Summed Sequences: 249/500, Accuracy: 49.80%
Number of Correctly Summed Sequences: 32/1000, Accuracy: 3.20%


### RNNs

Recurrent Neural Networks (RNNs) are a type of neural network designed for sequence data. Unlike standard feed-forward neural networks, RNNs have **recursion** in their architecture, which allows them to maintain a form of memory, known as **hidden state** across different time steps. Can you think why they might suit our addition problem?



In a vanilla RNN, the recursion can be mathematically represented as follows:

\begin{align}
\boldsymbol{h}^{\{t\}} = f\left(\boldsymbol{W}_{hh} \cdot \boldsymbol{h}^{\{t-1\}}  + \boldsymbol{W}_{hx} \cdot \boldsymbol{x}^{\{t\}}  + \boldsymbol{b}_h \right)
\end{align}

Where:
- $\boldsymbol{h}^{\{t\}}$ is the hidden state at time step $t$.
- $\boldsymbol{h}^{\{t-1\}}$ is the hidden state from the previous time step.
- $\boldsymbol{x}^{\{t\}}$ is the input at time step $t$.
- $\boldsymbol{W}_{hh}$ and $\boldsymbol{W}_{hx}$ are weights of the model, and $\boldsymbol{b}_h$ is the bias term.
- $f$ is the activation function (commonly $ \tanh $ or $ \text{ReLU} $).

<div style="text-align: center;">
    <img src="data/ece4179_ws10_rnn1.png" alt="RNN Example1" style="width: 50%;"/>
</div>




To understand how an RNN process sequences, you can think of it as unrolling the recursion over time. The hidden state at each time step is updated based on the input at that time step and the hidden state from the previous time step. This allows the RNN to maintain a form of memory across different time steps.

<div style="text-align: center;">
    <img src="data/ece4179_ws10_rnn2.png" alt="RNN Example2" style="width: 50%;"/>
</div>






### LSTM in PyTorch



A **Long Short-Term Memory (LSTM)** network is a type of recurrent neural network (RNN) designed to handle sequence data and learn long-term dependencies. Unlike a vanilla RNN, an LSTM contains **cell states** and **gates** (e.g., forget) to control the flow of information and prevent issues like vanishing gradients.

In PyTorch, an LSTM layer processes an input sequence and returns two outputs: 
1. The output for each time step.
2. The hidden and cell states.

The input to the LSTM layer in PyTorch has the following shape:

\begin{align*}
    \text{input} = (\text{B}, \tau, \text{input\_size})
\end{align*}

- $\text{B}$ is the **batch size**, the number of sequences processed simultaneously.
- $\tau$ is the **sequence length**, which is the number of time steps in each sequence.
- $\text{input\_size}$ refers to the size (number of features) of each element in the sequence.

For example, in a problem where each element in the sequence is a 2D vector (two features), and we process a batch of 4 sequences of length 5 in one batch, the input will have shape `(4, 5, 2)`.

LSTMs maintain a **hidden state** and a **cell state** for each layer. The hidden and cell states have the following shape:

\begin{align*}
    \text{hidden}, \text{cell} = (\text{num\_layers}, \text{B}, \text{hidden\_size})
\end{align*}

Where:
- $\text{num\_layers}$ is the number of LSTM layers.
- $ \text{hidden\_size} $ is the size of the hidden state (number of features in the hidden state).

If your LSTM has 2 layers, a batch size of 4, and a hidden size of 128, the hidden and cell states will have shape `(2, 4, 128)`.



The output of the LSTM consists of:
1. **Output for each time step**: This is the hidden state for each time step in the sequence.
   \begin{align*}
        \text{output} = (\text{B}, \tau, \text{hidden\_size})\;.
   \]
   
   If you process 4 sequences of length 5, and the hidden size is 128, the output will have the shape `(4, 5, 128)`.

2. **Final hidden and cell states**:
   \begin{align*}
   \text{hidden}, \text{cell} = (\text{num\_layers}, \text{B}, \text{hidden\_size})
   \end{align*}
   These tensors contain the final hidden and cell states for each layer and for each sequence in the batch. If the LSTM has 2 layers, a batch size of 4, and a hidden size of 128, both the hidden and cell states will have shape `(2, 4, 128)`.


### Did You Know? 

Very recently, advancements in LSTM architecture have led to the development of **xLSTM: Extended Long Short-Term Memory**. This new variation introduces modifications to the standard LSTM, enhancing its ability to **retain memory** over longer sequences and improve its ability to learn **long-range dependencies**. 

While we’ll be working with the traditional LSTM in this notebook, it's exciting to know that LSTMs are continuously evolving, and innovations like xLSTM are pushing the boundaries of what these models can achieve. 


<div style="text-align: center;">
    <img src="data/xlstm.jpg" alt="xLSTM" style="width: 50%;"/>
</div>

In [9]:
class RNNAddition(nn.Module):
    def __init__(self, hidden_size=128, seq_len=3, num_layers=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.num_layers = num_layers  # Number of LSTM layers
        
        # Define the LSTM layer with num_layers=2
        self.lstm = nn.LSTM(input_size=2, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

        # Classification head for each digit position
        self.fc = nn.Linear(hidden_size, 10)  # Output size is 10 (for digits 0-9)

    def init_hidden(self, batch_size, device):
        """
        Initializes the hidden state and cell state for the LSTM dynamically based on LSTM configuration.
        
        Args:
            batch_size (int): The size of the batch.
            device (torch.device): The device (CPU or GPU).
        
        Returns:
            tuple: The initial hidden state and cell state for the LSTM.
        """
        # Dynamically get the number of layers and hidden size from the LSTM configuration
        num_layers = self.lstm.num_layers
        hidden_size = self.lstm.hidden_size

        # LSTM requires both hidden and cell state for each layer
        hidden = torch.zeros(num_layers, batch_size, hidden_size).to(device)
        cell = torch.zeros(num_layers, batch_size, hidden_size).to(device)
        return hidden, cell

    def forward(self, seq1, seq2):
        batch_size = seq1.size(0)  # Get the batch size
        device = seq1.device  # Get the device (CPU or GPU)

        # Initialize hidden state (and cell state) for the LSTM at the start of the batch
        hidden = self.init_hidden(batch_size, device)

        # Reverse the sequences (from right to left)
        seq1 = torch.flip(seq1, dims=[1])  # Reverse seq1 along the sequence length dimension
        seq2 = torch.flip(seq2, dims=[1])  # Reverse seq2 along the sequence length dimension
        
        # Combine the two sequences by stacking them along a new dimension
        x = torch.stack((seq1, seq2), dim=2).float()  # Shape: [batch_size, seq_len, 2] (input_size=2)
        
        # Run the sequences through the LSTM
        rnn_out, (hidden, cell) = self.lstm(x, hidden)

        # Reverse the output to match the original left-to-right sequence
        rnn_out = torch.flip(rnn_out, dims=[1])  # Reverse rnn_out along the sequence length dimension
        
        # Apply the classification head to each time step
        output = self.fc(rnn_out)  # Shape: [batch_size, seq_len, 10]
        
        # Permute the output to [batch_size, 10, seq_len] for CrossEntropyLoss
        return output.permute(0, 2, 1)  # Shape: [batch_size, 10, seq_len]


In [10]:

hidden_size = 128  # Hidden size of the RNN


rnn_addition_model = RNNAddition(hidden_size=hidden_size, num_layers=2) 

# Assuming train_loader is already created with the DigitAdditionDataset
train_addition(rnn_addition_model, train_loader, num_epochs=500, lr=0.01)

Training Progress: 100%|██████████| 500/500 [00:36<00:00, 13.80it/s, loss=0.00221]


In [11]:
# evaluate the RNN model on the test dataset
acc_trn_rnn_addition = evaluate_addition(rnn_addition_model, train_loader)
acc_tst_rnn_addition = evaluate_addition(rnn_addition_model, test_loader)

Number of Correctly Summed Sequences: 500/500, Accuracy: 100.00%
Number of Correctly Summed Sequences: 754/1000, Accuracy: 75.40%


### Language Models

A **language model (LM)** is a type of model that learns to predict the next token in a sequence, given a preceding context. An example of an LM, duh, ChatGPT! So, let's design one 

### The Mysterious Island

**Jules Verne** (1828–1905) was a French novelist, and poet, widely regarded as one of the pioneers of science fiction. His adventure novels, filled with detailed scientific explanations. Some of his most famous works include **"Twenty Thousand Leagues Under the Sea"**, **"Around the World in Eighty Days"**, and **"Journey to the Center of the Earth"**.

**"The Mysterious Island"** is the story of five castaways who find themselves stranded on a mysterious, uninhabited island. Using their ingenuity and resourcefulness, they attempt to survive, all while uncovering the secrets of the island. In this task, we’ll use **"The Mysterious Island"** as the training data to build an **LM**, maybe we can mimic Verne’s unique writing style.






<div style="text-align: center;">
    <img src="data/Jules_Verne.jpg" alt="Jules Verne" style="width: 25%;"/>
</div>

In [12]:
## Reading and processing text
with open("data/The_Mysterious_Island.txt", "r", encoding="utf8") as jv_file:
    jv_book_text = jv_file.read()

# Create a set of unique characters
jv_char_set = set(jv_book_text)

# Print results
print('Total Length:', len(jv_book_text))
print('Unique Characters:', len(jv_char_set))

Total Length: 1131713
Unique Characters: 88


### Tokenization

A tokenizer processes text and converts it to a sequence of symbols or tokens that can be fed into a model. In this task, we’ll use a simple tokenizer that converts the text to lowercase and splits it into words. The **Tokenizer** class is responsible for:
1. **Creating a Vocabulary**: The tokenizer identifies all unique characters in the input text and builds a vocabulary, which is a sorted list of these unique characters.
2. **Character-to-Integer Mapping**: Once the vocabulary is established, the tokenizer creates a dictionary that maps each character to a unique integer index (`char2int`). This enables the model to work with numbers rather than raw text.
3. **Integer-to-Character Mapping**: The tokenizer also builds the reverse mapping (`int2char`), which allows the model to convert predicted integers back into characters for generating readable text.


- `encode(text)`: This method converts input text into a list of integer values based on the character-to-integer mapping. The output is a NumPy array of integers, where each integer represents a character in the text.
  
  Example:
  ```python
  tokenizer.encode('Hello') 
  # Output: [8, 4, 11, 11, 14]  (based on the vocabulary and char2int mapping)
  ```

- `decode(encoded_text)`: This method converts a list of integers (representing characters) back into the original text using the integer-to-character mapping.
  
  Example:
  ```python
  tokenizer.decode([8, 4, 11, 11, 14]) 
  # Output: 'Hello'  (converts integers back to the corresponding characters)
  ```


BTW, check this out [OpenAI Tokenizer](https://platform.openai.com/tokenizer)! 🤖


In [13]:
class Tokenizer:
    def __init__(self, text):
        # Create the vocabulary (sorted set of unique characters)
        self.vocab = sorted(set(text))
        self.char2int = {ch: i for i, ch in enumerate(self.vocab)}
        self.int2char = np.array(self.vocab)
    
    def encode(self, text):
        """Converts text into an array of integers."""
        return np.array([self.char2int[ch] for ch in text], dtype=np.int32)
    
    def decode(self, encoded_text):
        """Converts an array of integers back into text."""
        return ''.join(self.int2char[encoded_text])

<div style="background-color: #2b0080; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #3. Study the character-level tokenizer</span>

To have a short and fun break, study the `Tokenizer` class provided below. Understand how it works and how it can be used to encode and decode text. Feel free to experiment with the tokenizer by encoding and decoding different texts.

</div>

In [14]:
# Initialize the Tokenizer with the text
jv_tokenizer = Tokenizer(jv_book_text)

# Provide a custom text for encoding and decoding
prompt = "hello ece4179"

# Encode the custom text
enc_tokens = jv_tokenizer.encode(prompt)

# Print the encoded custom text
print('Prompt:', prompt)
print('Encoded tokens:', enc_tokens)

# Decode the encoded custom text back to the original text
dec_prompt = jv_tokenizer.decode(enc_tokens)

# Print the decoded text
print('Decoded prompt from tokens:', dec_prompt)

Prompt: hello ece4179
Encoded tokens: [64 61 68 68 71  1 61 59 61 18 15 21 23]
Decoded prompt from tokens: hello ece4179


### Training data

The **TextDataset** class is responsible for preparing the training data for our LM. It encodes the input text into integer sequences and creates pairs of input-target data that the model uses to learn how to predict the next character in a sequence.


When training a character-level LM, the goal is to feed sequences of characters (input) into the model and have it predict the next character (target). The **TextDataset** class constructs the training data by creating sliding windows of sequences from the encoded text.

The main idea is to slide a window of `seq_length` characters across the encoded text. For each index idx, the dataset generates an input sequence and its corresponding target sequence. The input sequence is a chunk of seq_length characters starting at position idx in the encoded text. The target sequence is created by shifting the input sequence by one character. This teaches the model to predict the next character for each position in the input sequence.


In [15]:
class TextDataset(Dataset):
    def __init__(self, book_text, tokenizer, seq_length):
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        # Encode the entire book text using the tokenizer
        self.text_encoded = self.tokenizer.encode(book_text)
    
    def __len__(self):
        # The number of available chunks in the dataset
        return len(self.text_encoded) - self.seq_length

    def __getitem__(self, idx):
        # Generate input sequence and target dynamically
        input_seq = self.text_encoded[idx:idx + self.seq_length]
        target_char = self.text_encoded[idx+1:idx + self.seq_length+1]
        return torch.tensor(input_seq).long(), torch.tensor(target_char).float()

In [16]:
seq_length = 40  # Sequence length for training
jv_dataset = TextDataset(jv_book_text, jv_tokenizer, seq_length=seq_length)

batch_size = 64
jv_dataloader = DataLoader(jv_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# let's print the first sample from the dataset
trn_x_sample, trn_y_sample = jv_dataset[0]
print(f'Input sequence: {trn_x_sample}')
print(f'Target character: {trn_y_sample}')

Input sequence: tensor([87, 48, 64, 61,  1, 44, 74, 71, 66, 61, 59, 76,  1, 35, 77, 76, 61, 70,
        58, 61, 74, 63,  1, 33, 30, 71, 71, 67,  1, 71, 62,  1, 48, 64, 61,  1,
        41, 81, 75, 76])
Target character: tensor([48., 64., 61.,  1., 44., 74., 71., 66., 61., 59., 76.,  1., 35., 77.,
        76., 61., 70., 58., 61., 74., 63.,  1., 33., 30., 71., 71., 67.,  1.,
        71., 62.,  1., 48., 64., 61.,  1., 41., 81., 75., 76., 61.])


<div style="background-color: #2b0080; color: white; padding: 10px; border-radius: 5px;">

### <span style="color: pink;">Task #4. Create an RNN for Character-Level LM</span>

Your RNN should consist of the following components:

1. **Embedding Layer**: This layer will map each character (represented as an integer) into a dense vector of fixed size (`embed_dim`). The embedding allows the model to learn meaningful representations of each character.

2. **LSTM Layer**: The core of your RNN model will be an **LSTM** (Long Short-Term Memory) layer. This layer will process the input sequences and capture patterns over time.

3. **Fully Connected (Linear) Layer**: After processing the sequence with the LSTM, a fully connected layer will map the hidden states back to the vocabulary space, allowing the model to predict the next character.

</div>

In [17]:
class jv_RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) 
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x)
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size)
        cell = torch.zeros(1, batch_size, self.rnn_hidden_size)
        return hidden.to(device), cell.to(device)
    


In [18]:
vocab_size = len(jv_tokenizer.vocab)
embed_dim = 256
rnn_hidden_size = 512

# instantiate the model
jv_model = jv_RNN(vocab_size, embed_dim, rnn_hidden_size) 
jv_model = jv_model.to(device)
jv_model

jv_RNN(
  (embedding): Embedding(88, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=88, bias=True)
)

### Training Loop for LM

The training loop for the LM is similar to the training loop for the previous task to a great degree. The loss function used in the training loop is the **CrossEntropyLoss**. We will train the model for fixed number of iterations in the workshop, feel free to improve the model by training it for more.

In [19]:


# Loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(jv_model.parameters(), lr=0.001)

# Number of iterations
num_iterations = 10000
print_every_k = 100  # Print and update the progress bar every `k` iterations
batch_size = 64  # Example batch size


# Initialize the progress bar for iterations
pbar = tqdm(total=num_iterations, desc="Training Progress", position=0, leave=True)


total_loss = 0

for iteration in range(num_iterations):

    seq_batch, target_batch = next(iter(jv_dataloader))
    
    # Initialize hidden and cell states
    hidden, cell = jv_model.init_hidden(batch_size)

    # Move batch to the correct device (CPU/GPU)
    seq_batch, target_batch = seq_batch.to(device), target_batch.to(device)

    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass through the model with the entire sequence
    pred, hidden, cell = jv_model(seq_batch, hidden, cell)
    pred = pred.view(-1, vocab_size)  # Reshape to [batch_size,  seq_length, vocab_size]
    # Compute loss for the last time step prediction
    loss = loss_fn(pred, target_batch.view(-1).long())

    # Backpropagation and optimization
    loss.backward()
    optimizer.step()

    # Accumulate total loss for this iteration
    total_loss += loss.item()

    # Update the progress bar and print every `k` iterations
    if iteration % print_every_k == 0:
        # Calculate the average loss over the last `k` iterations
        avg_loss = total_loss / print_every_k
        
        # Print and update tqdm
        # print(f"Iteration {iteration}/{num_iterations}, Loss: {avg_loss:.4f}")
        
        # Update the progress bar with the average loss
        pbar.set_postfix(loss=avg_loss)
        pbar.update(print_every_k)  # Update the progress bar by 'k' steps
        
        # Reset total_loss for the next `k` iterations
        total_loss = 0

# Close the progress bar when training is done
pbar.close()

Training Progress: 100%|██████████| 10000/10000 [11:34<00:00, 14.41it/s, loss=1.03] 


### Write with your model

In [20]:
from torch.distributions import Categorical

def sample(model, starting_str, len_generated_text=500, temperature=1.0):
    # Encode the starting string
    encoded_input = jv_tokenizer.encode(starting_str)
    encoded_input = torch.tensor(encoded_input).unsqueeze(0)  # Shape [1, seq_length]

    generated_str = starting_str

    # Set model to evaluation mode
    model.eval()
    hidden, cell = model.init_hidden(1)  # For batch size of 1
    hidden, cell = hidden.to('cpu'), cell.to('cpu')

    # Pass the entire starting string through the model to prime the hidden state
    _, hidden, cell = model(encoded_input, hidden, cell)

    # Start sampling from the last character of the input
    last_char = encoded_input[:, -1]  # Get the last character of the input

    # Generate the next characters
    for i in range(len_generated_text):
        # Forward pass for the last character
        logits, hidden, cell = model(last_char.view(1,1), hidden, cell)  # Shape [1, 1]
        logits = torch.squeeze(logits, 0)  # Remove batch dimension

        # Scale logits by temperature (divide by temperature)
        scaled_logits = logits / temperature

        # Sample from the scaled distribution
        m = Categorical(logits=scaled_logits)
        last_char = m.sample()

        # Decode and append the generated character to the string
        generated_str += jv_tokenizer.decode([last_char.item()])

    return generated_str


In [22]:

jv_model.to('cpu')
print(sample(jv_model, starting_str='The island', len_generated_text=500, temperature=0.50))

The islanders of the water would have been contained on the shore of the Mercy. The colonists was difficult to do the return of this species of a few days they had taken principal of the island.

It was not a straight and southern point of the island, and the colonists then to see the convicts could not be used in a corner of a fire which we will not be surprised by the convicts and the coast, and the two companions but the engineer had not been able to arrest to the forest. The poor of the island was not
